In [ ]:
import parflow as pf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#read in the mask for the domain
pf_output_dir = 'path/to/parflow/run/outputs'
runname = '<parflow_runname>'
file = f'{pf_output_dir}/{runname}.out.mask.pfb'

print(file)
mask = pf.read_pfb(file)
mask[mask ==0] = False
mask[mask ==99999] = True
print(np.shape(mask))
print(np.nanmax(mask))
print(np.nanmin(mask))

#plot mask to check
plt.figure()
plt.imshow(mask[0,:,:], origin='lower', cmap='plasma',interpolation = "none")
plt.axis('off')
plt.savefig('mask.png', dpi=500,bbox_inches='tight')

In [ ]:
# scan and check ParFlow-CLM forcing files
# use the mask to filter values and check for NaNs in the masked area
# print out file name and timestep with missing values
vars = ['APCP', 'DLWR', 'DSWR', 'Temp', 'SPFH', 'UGRD', 'VGRD', 'Press']
forcing_dir = 'path/to/forcing/inputs/for/run'
dt = 24

#step through the files and check the time range over a year and for each variable in the list above

# set start / finish range, note that these need to be forcing file incriments, i.e. match the X_to_Y
# in the file name
begin_step = 1
end_step = 3048

# loop over variables, then timesteps
for var in vars:
    print(var)
    for i in range(begin_step,end_step,dt):
        file_name = 'CW3E.'+var+"."+f"{i:06d}"+'_to_'+f"{(i+dt-1):06d}"+'.pfb'
        file_path = f'{forcing_dir}/{file_name}'
        check = pf.read_pfb(file_path)
        check[check < -1.0e+38] = np.nan
        for ii in range(24):
            nan_in_forcing = np.isnan(check[ii][mask[0]==True]).any()
            if nan_in_forcing:
                print("NaNs found file:",file_name," time:",i+ii," step:", ii)